In [1]:
import os
import zipfile

zip_path = r"C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\osm-pennsylvania-251116-free.shp.zip"

# Choose destination directory (this can be anywhere outside your repo)
extract_to = r"C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\OSM_PA"

# Create directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Confirm extraction
os.listdir(extract_to)

['gis_osm_buildings_a_free_1.cpg',
 'gis_osm_buildings_a_free_1.dbf',
 'gis_osm_buildings_a_free_1.prj',
 'gis_osm_buildings_a_free_1.shp',
 'gis_osm_buildings_a_free_1.shx',
 'gis_osm_landuse_a_free_1.cpg',
 'gis_osm_landuse_a_free_1.dbf',
 'gis_osm_landuse_a_free_1.prj',
 'gis_osm_landuse_a_free_1.shp',
 'gis_osm_landuse_a_free_1.shx',
 'gis_osm_natural_a_free_1.cpg',
 'gis_osm_natural_a_free_1.dbf',
 'gis_osm_natural_a_free_1.prj',
 'gis_osm_natural_a_free_1.shp',
 'gis_osm_natural_a_free_1.shx',
 'gis_osm_natural_free_1.cpg',
 'gis_osm_natural_free_1.dbf',
 'gis_osm_natural_free_1.prj',
 'gis_osm_natural_free_1.shp',
 'gis_osm_natural_free_1.shx',
 'gis_osm_places_a_free_1.cpg',
 'gis_osm_places_a_free_1.dbf',
 'gis_osm_places_a_free_1.prj',
 'gis_osm_places_a_free_1.shp',
 'gis_osm_places_a_free_1.shx',
 'gis_osm_places_free_1.cpg',
 'gis_osm_places_free_1.dbf',
 'gis_osm_places_free_1.prj',
 'gis_osm_places_free_1.shp',
 'gis_osm_places_free_1.shx',
 'gis_osm_pofw_a_free_1.cpg',


In [4]:
import osmnx as ox
import geopandas as gpd
import pandas as pd

tags = {"power": "substation"}

counties = [
    "Philadelphia County, Pennsylvania, USA",
    "Montgomery County, Pennsylvania, USA",
    "Delaware County, Pennsylvania, USA",
    "Bucks County, Pennsylvania, USA",
    "Chester County, Pennsylvania, USA",
]

gdfs = []
for place in counties:
    gdf_place = ox.features.features_from_place(place, tags=tags)
    gdf_place["source_place"] = place
    gdfs.append(gdf_place)

substations = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

len(substations), substations.head()


(298,
                                             geometry       power  \
 0                         POINT (-75.16031 39.96686)  substation   
 1  MULTIPOLYGON (((-75.18922 39.97831, -75.18813 ...  substation   
 2  MULTIPOLYGON (((-75.13495 40.03598, -75.13498 ...  substation   
 3  MULTIPOLYGON (((-75.15678 39.95916, -75.15694 ...  substation   
 4  POLYGON ((-75.16607 39.90784, -75.16511 39.907...  substation   
 
            substation                   name  \
 0  minor_distribution                    NaN   
 1        distribution      Master Substation   
 2        distribution       Tabor Substation   
 3                 NaN  Callowhill Substation   
 4  minor_distribution      Packer Substation   
 
                                                 note operator  \
 0                                                NaN      NaN   
 1                                                NaN     PECO   
 2                                                NaN     PECO   
 3                

In [6]:
cols_to_keep = [
    "name",
    "operator",
    "voltage",
    "substation",
    "geometry"
]

sub_clean = substations[cols_to_keep].copy()
sub_clean.head()

# Split semicolon-separated voltages into numeric fields
sub_clean["voltage_list"] = sub_clean["voltage"].str.split(";")


In [7]:
sub_clean.head()

,name,operator,voltage,substation,geometry,voltage_list
0,NaN,NaN,NaN,minor_distribution,POINT (-75.16031 39.96686),NaN
1,Master Substation,PECO,230000;138000;69000,distribution,"MULTIPOLYGON (((-75.18922 39.97831, -75.18813 ...","[230000, 138000, 69000]"
2,Tabor Substation,PECO,230000;69000,distribution,"MULTIPOLYGON (((-75.13495 40.03598, -75.13498 ...","[230000, 69000]"
3,Callowhill Substation,PECO,138000,NaN,"MULTIPOLYGON (((-75.15678 39.95916, -75.15694 ...",[138000]
4,Packer Substation,PECO,69000,minor_distribution,"POLYGON ((-75.16607 39.90784, -75.16511 39.907...",[69000]


In [9]:
# Step 1: Reproject to PA projected CRS
sub_proj = sub_clean.to_crs(2272)

# Step 2: Compute centroids in projected CRS
sub_centroids = sub_proj.copy()
sub_centroids["geometry"] = sub_proj.geometry.centroid


In [10]:
sub_proj.crs
sub_centroids.head()


,name,operator,voltage,substation,geometry,voltage_list
0,NaN,NaN,NaN,minor_distribution,POINT (2694278.613 241426.895),NaN
1,Master Substation,PECO,230000;138000;69000,distribution,POINT (2686292.516 245575.081),"[230000, 138000, 69000]"
2,Tabor Substation,PECO,230000;69000,distribution,POINT (2700364.179 266651.128),"[230000, 69000]"
3,Callowhill Substation,PECO,138000,NaN,POINT (2695212.95 238881.483),[138000]
4,Packer Substation,PECO,69000,minor_distribution,POINT (2693414.976 219769.854),[69000]


In [11]:
len(sub_centroids), sub_centroids.head()


(298,
                     name operator              voltage          substation  \
 0                    NaN      NaN                  NaN  minor_distribution   
 1      Master Substation     PECO  230000;138000;69000        distribution   
 2       Tabor Substation     PECO         230000;69000        distribution   
 3  Callowhill Substation     PECO               138000                 NaN   
 4      Packer Substation     PECO                69000  minor_distribution   
 
                          geometry             voltage_list  
 0  POINT (2694278.613 241426.895)                      NaN  
 1  POINT (2686292.516 245575.081)  [230000, 138000, 69000]  
 2  POINT (2700364.179 266651.128)          [230000, 69000]  
 3   POINT (2695212.95 238881.483)                 [138000]  
 4  POINT (2693414.976 219769.854)                  [69000]  )

In [12]:
data_root = r"C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data"
osm_dir   = os.path.join(data_root, "OSM_PA")
os.makedirs(osm_dir, exist_ok=True)

sub_path = os.path.join(osm_dir, "substations_sepa_clean.gpkg")
sub_centroids.to_file(sub_path, layer="substations_sepa", driver="GPKG")

sub_path

'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\OSM_PA\\substations_sepa_clean.gpkg'

In [13]:
zip_path_frd = os.path.join(
    data_root,
    "FRD_02040202_PA_Shapefiles_20160801.zip"
)

frd_dir = os.path.join(data_root, "FRD_02040202_PA_Shapefiles_20160801")
os.makedirs(frd_dir, exist_ok=True)

with zipfile.ZipFile(zip_path_frd, "r") as z:
    z.extractall(frd_dir)

os.listdir(frd_dir)

['02040202_PA_FRD_metadata.xml',
 'FRD_Model_Info.cpg',
 'FRD_Model_Info.dbf',
 'FRD_Model_Info.dbf.xml',
 'FRD_Study_Info.cpg',
 'FRD_Study_Info.dbf',
 'FRD_Study_Info.dbf.xml',
 'FRR_Custom.cpg',
 'FRR_Custom.dbf',
 'FRR_Custom.dbf.xml',
 'FRR_Images.cpg',
 'FRR_Images.dbf',
 'FRR_Images.dbf.xml',
 'FRR_Project.cpg',
 'FRR_Project.dbf',
 'FRR_Project.dbf.xml',
 'L_Claims.cpg',
 'L_Claims.dbf',
 'L_Claims.dbf.xml',
 'L_CSLF_Summary.cpg',
 'L_CSLF_Summary.dbf',
 'L_CSLF_Summary.dbf.xml',
 'L_Exposure_TEIF1.cpg',
 'L_Exposure_TEIF1.dbf',
 'L_Exposure_TEIF1.dbf.xml',
 'L_RA_AAL.cpg',
 'L_RA_AAL.dbf',
 'L_RA_AAL.dbf.xml',
 'L_RA_Summary_TEIF1.cpg',
 'L_RA_Summary_TEIF1.dbf',
 'L_RA_Summary_TEIF1.dbf.xml',
 'L_RA_TEIF1.cpg',
 'L_RA_TEIF1.dbf',
 'L_RA_TEIF1.dbf.xml',
 'L_Source_Cit.cpg',
 'L_Source_Cit.dbf',
 'L_Source_Cit.dbf.xml',
 'S_Carto_Ar.cpg',
 'S_Carto_Ar.dbf',
 'S_Carto_Ar.prj',
 'S_Carto_Ar.sbn',
 'S_Carto_Ar.sbx',
 'S_Carto_Ar.shp',
 'S_Carto_Ar.shp.xml',
 'S_Carto_Ar.shx',
 'S_

In [14]:
import geopandas as gpd
import glob

# list shapefiles to confirm the exact hazard file name
glob.glob(os.path.join(frd_dir, "*.shp"))


['C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_Carto_Ar.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_Carto_Ln.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_Carto_Pt.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_CenBlk_Ar_TEIF1.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_CSLF_Ar.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_FRD_Pol_Ar.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\Data\\FRD_02040202_PA_Shapefiles_20160801\\S_FRD_Proj_Ar.shp',
 'C:\\Users\\Owner\\Downloads\\Geospatial Data Science\\Final Project\\D

In [16]:
data_root = r"C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data"

# List of county NFHL zip files
nfhl_zips = [
    os.path.join(data_root, "philadelphia-county-nfhl.zip"),
    os.path.join(data_root, "delco-nfhl.zip"),
    os.path.join(data_root, "montco-nfhl.zip"),
    os.path.join(data_root, "bucks-nfhl.zip"),
    os.path.join(data_root, "chesterco-nfhl.zip")
]

# Unzip destination directory
nfhl_root = os.path.join(data_root, "NFHL")
os.makedirs(nfhl_root, exist_ok=True)

# Unzip each county
for zip_path in nfhl_zips:
    county_name = os.path.splitext(os.path.basename(zip_path))[0]
    county_dir = os.path.join(nfhl_root, county_name)
    os.makedirs(county_dir, exist_ok=True)

    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(county_dir)

    print(f"Extracted: {county_name}")

Extracted: philadelphia-county-nfhl
Extracted: delco-nfhl
Extracted: montco-nfhl
Extracted: bucks-nfhl
Extracted: chesterco-nfhl


In [18]:
for county_folder in os.listdir(nfhl_root):
    full_folder = os.path.join(nfhl_root, county_folder)
    shp_files = glob.glob(os.path.join(full_folder, "*.shp"))

    print("\n", county_folder)
    for shp in shp_files:
        print("   ", os.path.basename(shp))



 bucks-nfhl
    S_BASE_INDEX.shp
    S_BFE.shp
    S_FIRM_PAN.shp
    S_FLD_HAZ_AR.shp
    S_FLD_HAZ_LN.shp
    S_GEN_STRUCT.shp
    S_LABEL_LD.shp
    S_LABEL_PT.shp
    S_LEVEE.shp
    S_LOMR.shp
    S_POL_AR.shp
    S_PROFIL_BASLN.shp
    S_STN_START.shp
    S_SUBMITTAL_INFO.shp
    S_WTR_LN.shp
    S_XS.shp

 chesterco-nfhl
    S_BASE_INDEX.shp
    S_BFE.shp
    S_FIRM_PAN.shp
    S_FLD_HAZ_AR.shp
    S_FLD_HAZ_LN.shp
    S_GAGE.shp
    S_GEN_STRUCT.shp
    S_LABEL_LD.shp
    S_LABEL_PT.shp
    S_LEVEE.shp
    S_LOMR.shp
    S_NODES.shp
    S_POL_AR.shp
    S_PROFIL_BASLN.shp
    S_STN_START.shp
    S_SUBBASINS.shp
    S_SUBMITTAL_INFO.shp
    S_WTR_LN.shp
    S_XS.shp

 delco-nfhl
    S_BASE_INDEX.shp
    S_BFE.shp
    S_CST_TSCT_LN.shp
    S_FIRM_PAN.shp
    S_FLD_HAZ_AR.shp
    S_FLD_HAZ_LN.shp
    S_GEN_STRUCT.shp
    S_LABEL_LD.shp
    S_LABEL_PT.shp
    S_LEVEE.shp
    S_LIMWA.shp
    S_LOMR.shp
    S_POL_AR.shp
    S_PROFIL_BASLN.shp
    S_STN_START.shp
    S_SUBMITTAL_INFO

In [19]:
# Loading county flood hazard polygons

data_root = r"C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data"
nfhl_root = os.path.join(data_root, "NFHL")

flood_layers = []

for county_folder in os.listdir(nfhl_root):
    county_path = os.path.join(nfhl_root, county_folder)
    flood_path = os.path.join(county_path, "S_FLD_HAZ_AR.shp")

    if os.path.exists(flood_path):
        print(f"Loading {flood_path}")
        gdf = gpd.read_file(flood_path)
        gdf["county"] = county_folder
        flood_layers.append(gdf)
    else:
        print(f"Warning: No S_FLD_HAZ_AR.shp found in {county_folder}")

Loading C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\NFHL\bucks-nfhl\S_FLD_HAZ_AR.shp
Loading C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\NFHL\chesterco-nfhl\S_FLD_HAZ_AR.shp
Loading C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\NFHL\delco-nfhl\S_FLD_HAZ_AR.shp
Loading C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\NFHL\montco-nfhl\S_FLD_HAZ_AR.shp
Loading C:\Users\Owner\Downloads\Geospatial Data Science\Final Project\Data\NFHL\philadelphia-county-nfhl\S_FLD_HAZ_AR.shp


In [20]:
flood_all = gpd.GeoDataFrame(
    pd.concat(flood_layers, ignore_index=True),
    crs=flood_layers[0].crs
)